# Technical Indicators Model
There are many technical indicators available to analyse stock prices. For this project, we will be focussing on the Simple Moving Average, Exponential Moving Average, Moving Average Convergence/Divergence and Relative Strength Index.

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sb
from yahoo_fin import stock_info as sf
sb.set()

In [2]:
stock = 'AAPL'
short_window = 20
medium_window = 50
start_date = '2017-03-01'
end_date = '2023-03-01'

## Get Stock and Indicator Data
For this project, I would be focussing on short term (20 days) to medium term (50 days) investing or trading, and would have to identify appropriate buy or sell positions to maximize my profits. Hence, I will require past stock data on AAPL from 2017-03-01 to 2023-03-01. I will also require the short and medium term technical indicator data for the SMA, EMA, MACD and RSI.

In [3]:
def getIndicator(indicator, data):
    if indicator.lower() == 'sma':
        for window in [short_window, medium_window]:
            data[f'sma_{window}'] = data['close'].rolling(window).mean()
    elif indicator.lower() == 'ema':
        for window in [short_window, medium_window]:
            data[f'ema_{window}'] = data['close'].ewm(span=window).mean()
    elif indicator.lower() == 'macd':
        for window in [12, 26]:
            data[f'macd_{window}'] = data['close'].ewm(span=window).mean()
        data['macd_value'] = data['macd_12'] - data['macd_26']
        data.drop(labels=['macd_12', 'macd_26'], axis=1, inplace=True)
        data['macd_signal'] = data['macd_value'].ewm(span=9).mean()
    return pd.DataFrame(data)

In [4]:
stock_df = sf.get_data(ticker=stock, start_date=start_date, end_date=end_date, index_as_date=False)
stock_df = getIndicator('sma', stock_df)
stock_df = getIndicator('ema', stock_df)
stock_df = getIndicator('macd', stock_df)
stock_df = stock_df[200:].sort_values(by='date', ascending=False).reset_index(drop=True)
stock_df.to_csv('AAPL_historical_data.csv')
stock_df

,date,open,high,low,close,adjclose,volume,ticker,sma_20,sma_50,ema_20,ema_50,macd_value,macd_signal
0,2023-02-28,147.050003,149.080002,146.830002,147.410004,147.410004,50547000,AAPL,150.634499,140.696199,148.507142,145.260747,1.678721,2.828329
1,2023-02-27,147.710007,149.169998,147.449997,147.919998,147.919998,44998500,AAPL,150.413999,140.612199,148.622630,145.173023,2.002871,3.115731
2,2023-02-24,147.110001,147.190002,145.720001,146.710007,146.710007,55469600,AAPL,150.314499,140.563199,148.696591,145.060901,2.335545,3.393946
3,2023-02-23,150.089996,150.339996,147.240005,149.399994,149.399994,48394200,AAPL,150.176999,140.518799,148.905705,144.993591,2.849385,3.658546
4,2023-02-22,148.869995,149.949997,147.160004,148.910004,148.910004,51011300,AAPL,149.799999,140.373999,148.853675,144.813738,3.189897,3.860837
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,2017-12-19,43.757500,43.847500,43.522499,43.634998,41.429256,109745600,AAPL,43.044250,41.980950,42.992976,42.106255,0.363827,0.320978
1306,2017-12-18,43.720001,44.299999,43.715000,44.105000,41.875500,117684400,AAPL,42.987250,41.887450,42.925394,42.043839,0.347791,0.310266
1307,2017-12-15,43.407501,43.542500,43.115002,43.492500,41.293961,160677200,AAPL,42.908875,41.781850,42.801225,41.959685,0.274410,0.300884
1308,2017-12-14,43.099998,43.282501,42.912498,43.055000,40.878574,81906000,AAPL,42.873000,41.688950,42.728460,41.897102,0.239122,0.307503


## Relationship between SMA and AAPL Stock Price
From the CandleStick chart below, we can observe that in general:
- Both the SMA 20 and the SMA 50 follow the rise and fall of AAPL's stock price
- The SMA 20 is more responsive to price changes compared to the SMA 50
- AAPL stock price rises when the SMA 20 rises above the SMA 50
- AAPL stock price falls when the SMA 20 falls below the SMA 50

Hence, a possible strategy that we can use to identify the buy and sell signals for AAPL stock is:
- Buy when the SMA 20 just rises above the SMA 50
- Sell when the SMA 20 just falls below the SMA 50

In [5]:
def plotMA(data, indicator):
    fig = go.Figure(
        data = [
            go.Candlestick(
                x = data['date'],
                low = data['low'],
                high = data['high'],
                open = data['open'],
                close = data['close'],
                name = stock.upper(),
            ),
        ],
    )
    window = {
        short_window: '#0062ff',
        medium_window: '#6e00b8',
    }
    for item in window:
        fig.add_trace(
            go.Scatter(
                x = data['date'],
                y = data[f'{indicator.lower()}_{item}'],
                line = {
                    'color': window[item],
                    'width': 2,
                },
                name = f'{indicator.upper()} {item}',
            )
        )
    fig.update_layout(xaxis_rangeslider_visible=False, title_text=f'Relationship between AAPL Stock Price and {indicator.upper()}')
    fig.update_xaxes(title_text='Date')
    fig.update_yaxes(title_text='Price')
    return fig

def plotMACD(data):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_candlestick(
        x = data['date'],
        low = data['low'],
        high = data['high'],
        open = data['open'],
        close = data['close'],
        name = stock.upper(),
    )
    window = {
        'value': '#0062ff',
        'signal': '#6e00b8',
    }
    for item in window:
        fig.add_trace(
            go.Scatter(
                x = data['date'],
                y = data[f'macd_{item}'],
                line = {
                    'color': window[item],
                    'width': 2,
                },
                name = f'MACD {item}',
            ),
            secondary_y=True,
        )
    fig.update_layout(xaxis_rangeslider_visible=False, title_text='Relationship between AAPL Stock Price and MACD')
    fig.update_xaxes(title_text='Date')
    fig.update_yaxes(title_text='Price', secondary_y=False)
    fig.update_yaxes(title_text='MACD', secondary_y=True)
    return fig

In [6]:
sma_fig = plotMA(stock_df, 'sma')
sma_fig.show()

### Create new columns for the stock price signal and position
With this observation and possible strategy, we can create 2 new columns, a 'signal' and a 'position' column that would aid us in our analysis:
- the 'signal' column captures whether or not the SMA 20 is greater than the SMA 50, and hence whether the AAPL stock appreciates or depreciates
    - 1 = SMA 20 > SMA 50, indicating that AAPL stock likely to appreciate
    - 0 = SMA 20 <= SMA 50, indicating that AAPL stock likely to depreciate
- The 'position' column captures the difference between the current day's signal and the previous day's signal
    - -1 = SMA 20 has just fallen below the SMA 50, so AAPL stock would likely depreciate since SMA 20 < SMA 50, hence should be a sell position 
    - 0 = No change in the likelihood of AAPL stock appreciating or depreciating, hence no buy or sell position
    - 1 = SMA 20 has just risen above the SMA 50, so AAPL stock would likely appreciate since SMA 20 > SMA 50, hence should be a buy position

In [7]:
sma_stock_df = stock_df.copy()
sma_stock_df['signal'] = np.where(sma_stock_df['sma_20'] > sma_stock_df['sma_50'], 1, 0)
sma_stock_df['position'] = sma_stock_df['signal'].diff()
sma_stock_df = sma_stock_df[1:].reset_index(drop=True)
sma_stock_df

,date,open,high,low,close,adjclose,volume,ticker,sma_20,sma_50,ema_20,ema_50,macd_value,macd_signal,signal,position
0,2023-02-27,147.710007,149.169998,147.449997,147.919998,147.919998,44998500,AAPL,150.413999,140.612199,148.622630,145.173023,2.002871,3.115731,1,0.0
1,2023-02-24,147.110001,147.190002,145.720001,146.710007,146.710007,55469600,AAPL,150.314499,140.563199,148.696591,145.060901,2.335545,3.393946,1,0.0
2,2023-02-23,150.089996,150.339996,147.240005,149.399994,149.399994,48394200,AAPL,150.176999,140.518799,148.905705,144.993591,2.849385,3.658546,1,0.0
3,2023-02-22,148.869995,149.949997,147.160004,148.910004,148.910004,51011300,AAPL,149.799999,140.373999,148.853675,144.813738,3.189897,3.860837,1,0.0
4,2023-02-21,150.199997,151.300003,148.410004,148.479996,148.479996,58867200,AAPL,149.480999,140.248799,148.847746,144.646543,3.628006,4.028571,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,2017-12-19,43.757500,43.847500,43.522499,43.634998,41.429256,109745600,AAPL,43.044250,41.980950,42.992976,42.106255,0.363827,0.320978,1,0.0
1305,2017-12-18,43.720001,44.299999,43.715000,44.105000,41.875500,117684400,AAPL,42.987250,41.887450,42.925394,42.043839,0.347791,0.310266,1,0.0
1306,2017-12-15,43.407501,43.542500,43.115002,43.492500,41.293961,160677200,AAPL,42.908875,41.781850,42.801225,41.959685,0.274410,0.300884,1,0.0
1307,2017-12-14,43.099998,43.282501,42.912498,43.055000,40.878574,81906000,AAPL,42.873000,41.688950,42.728460,41.897102,0.239122,0.307503,1,0.0


## Relationship between EMA and AAPL Stock Price
We can observe a similar trend to the SMA for the EMA in the Candlestick chart below, hence we can also follow a similar strategy to the one we used for the SMA

In [8]:
ema_fig = plotMA(stock_df, 'ema')
ema_fig.show()

### Create new columns for the stock price signal and position
From the CandleStick chart above, we can observe a similar trend to SMA, so we will create the same 'signal' and 'position' columns to aid us in our analysis

In [9]:
ema_stock_df = stock_df.copy()
ema_stock_df['signal'] = np.where(ema_stock_df['ema_20'] > ema_stock_df['ema_50'], 1, 0)
ema_stock_df['position'] = ema_stock_df['signal'].diff()
ema_stock_df = ema_stock_df[1:].reset_index(drop=True)
ema_stock_df

,date,open,high,low,close,adjclose,volume,ticker,sma_20,sma_50,ema_20,ema_50,macd_value,macd_signal,signal,position
0,2023-02-27,147.710007,149.169998,147.449997,147.919998,147.919998,44998500,AAPL,150.413999,140.612199,148.622630,145.173023,2.002871,3.115731,1,0.0
1,2023-02-24,147.110001,147.190002,145.720001,146.710007,146.710007,55469600,AAPL,150.314499,140.563199,148.696591,145.060901,2.335545,3.393946,1,0.0
2,2023-02-23,150.089996,150.339996,147.240005,149.399994,149.399994,48394200,AAPL,150.176999,140.518799,148.905705,144.993591,2.849385,3.658546,1,0.0
3,2023-02-22,148.869995,149.949997,147.160004,148.910004,148.910004,51011300,AAPL,149.799999,140.373999,148.853675,144.813738,3.189897,3.860837,1,0.0
4,2023-02-21,150.199997,151.300003,148.410004,148.479996,148.479996,58867200,AAPL,149.480999,140.248799,148.847746,144.646543,3.628006,4.028571,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,2017-12-19,43.757500,43.847500,43.522499,43.634998,41.429256,109745600,AAPL,43.044250,41.980950,42.992976,42.106255,0.363827,0.320978,1,0.0
1305,2017-12-18,43.720001,44.299999,43.715000,44.105000,41.875500,117684400,AAPL,42.987250,41.887450,42.925394,42.043839,0.347791,0.310266,1,0.0
1306,2017-12-15,43.407501,43.542500,43.115002,43.492500,41.293961,160677200,AAPL,42.908875,41.781850,42.801225,41.959685,0.274410,0.300884,1,0.0
1307,2017-12-14,43.099998,43.282501,42.912498,43.055000,40.878574,81906000,AAPL,42.873000,41.688950,42.728460,41.897102,0.239122,0.307503,1,0.0


### Relationship between MACD and AAPL Stock Price
If MACD Value > MACD Signal, buy signal, else sell signal

In [10]:
macd_fig = plotMACD(stock_df)
macd_fig.show()